In [17]:
from sklearn.preprocessing import KBinsDiscretizer # To turn continuous parameters of the cartpole environment into discrete values 
import numpy as np 
import time, math, random
from typing import Tuple

import gym # Requires pygame and Box2D packages

In [23]:
# Creating a CartPole environment
env = gym.make('CartPole-v1',render_mode="human")
# Goal is to have the cart move in a way that will allow the pole to balance itself upright

In [24]:
# Turning continuous values into discrete ones

n_bins = ( 6 , 12 )
lower_bounds = [ env.observation_space.low[2], -math.radians(50) ]
upper_bounds = [ env.observation_space.high[2], math.radians(50) ]

def discretizer( _ , __ , angle, pole_velocity ) -> Tuple[int,...]:
    """Convert continues state intro a discrete state"""
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bounds ])
    return tuple(map(int,est.transform([[angle, pole_velocity]])[0])) # Mostly based on the angle and velocity of the pole

In [25]:
# Initialize Q table
Q_table = np.zeros(n_bins + (env.action_space.n,))
Q_table.shape

(6, 12, 2)

In [26]:
# Choosing highest Q value
def policy( state : tuple ):
    """Choosing action based on epsilon-greedy policy"""
    return np.argmax(Q_table[state])

In [27]:
# Function for initializing new Q-values
def new_Q_value( reward : float ,  new_state : tuple , discount_factor=1 ) -> float:
    """Temperal diffrence for updating Q-value of state-action pair"""
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value

In [28]:
# Adaptive learning of Learning Rate
def learning_rate(n : int , min_rate=0.01 ) -> float  :
    """Decaying learning rate"""
    return max(min_rate, min(1.0, 1.0 - math.log10((n + 1) / 25)))

In [29]:

def exploration_rate(n : int, min_rate= 0.1 ) -> float :
    """Decaying exploration rate"""
    return max(min_rate, min(1, 1.0 - math.log10((n  + 1) / 25)))

In [ ]:
# Running 1000 episodes of q-learning 
n_episodes = 1000 
for e in range(n_episodes):
    print(env.reset()[0][0])
    # Discretize state into buckets
    current_state, done = discretizer(*env.reset()[0]), False
    
    while done==False:
        
        # policy action 
        action = policy(current_state) # exploit
        
        # insert random action
        if np.random.random() < exploration_rate(e) : 
            action = env.action_space.sample() # explore 
        
        print(env.step(action))

        # increment enviroment
        obs, reward, done, _, __= env.step(action)
        new_state = discretizer(*obs)
        
        # Update Q-Table
        lr = learning_rate(e)
        learnt_value = new_Q_value(reward , new_state )
        old_value = Q_table[current_state][action]
        Q_table[current_state][action] = (1-lr)*old_value + lr*learnt_value # Q-table updated
        
        current_state = new_state # Changing the current state
        
        # Render the cartpole environment (Box2D)
        env.render()
            
# Pole becomes more steady after multiple iterations
            